### Ingestion de la carpeta "production_company"

### Paso 1 - Leer los archivos CSV usando "DataFrameReader" de Spark

In [0]:
dbutils.widgets.text("p_enviroment", "")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.functions import current_timestamp,lit,col
from pyspark.sql.types import StructType,StructField,IntegerType,StringType


In [0]:
productions_companies_schema = StructType([
    StructField("companyId", IntegerType(), True),
    StructField("companyName", StringType(), True)
    ])


In [0]:
productions_companies_df = spark.read.schema(productions_companies_schema).csv(f"{bronze_folder_path}/{v_file_date}/production_company/")#Si tengo mas archivos en la carpeta y solo quiero cargar los del production_company deberia usar production_company_*.csv

In [0]:
productions_companies_df.printSchema()

In [0]:
display(productions_companies_df)

In [0]:
productions_companies_df.count()#Cuento los registros

#### Paso 2 - Renombrar las columnas y añadir nuevas columnas
1. "companyId" renombrar a "company_id"
2. "companyName" renombrar a "company_name"
3. Agregar las columnas "ingestion_date" y "enviroment"

In [0]:
productions_companies_final_df = add_ingestion_date(productions_companies_df).withColumnsRenamed({"companyId":"company_id","companyName":"company_name"}).withColumns({"enviroment":lit(v_enviroment),"file_date":lit(v_file_date)})

In [0]:
display(productions_companies_final_df)

#### Paso 4 - Escribir la salida en formato "Parquet"

In [0]:
merge_condition = 'tgt.company_id = src.company_id AND tgt.file_date = src.file_date'
merge_delta_lake(productions_companies_final_df, 'movie_silver', 'productions_companies', silver_folder_path, merge_condition, 'file_date')

In [0]:
%fs
ls /mnt/historialpeliculas/silver/productions_companies/

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.productions_companies
GROUP BY file_date;

In [0]:
dbutils.notebook.exit("Success")